In [1]:
import pandas as pd
import numpy as np
import pyodbc as pc
import sqlalchemy
import warnings

warnings.filterwarnings('ignore')

path_start = 'c:/tt/'
path_start_kntr = 'c:/tt/kntr/'

In [2]:
def connect():
    connection_string = "Driver=SQL Server;Server=localhost;Database=fabr;Trusted_Connection=Yes;"
    cnxn = pc.connect(connection_string, autocommit=True)
    return cnxn
def connect_sqlalchemy():
    engine = sqlalchemy.create_engine("mssql+pyodbc://DESKTOP-SFHJLGK/fabr?driver=ODBC+Driver+17+for+SQL+Server")
    return engine

def read_sql(cnxn, query):
    df1 = pd.read_sql(query, cnxn)
    return df1

def to_sql(cnxn, table, df):
    cur=cnxn.cursor()
    cols = ','.join(df.columns)
    vals = ','.join('?' * len(df.columns))
    query = f'''insert into {table}({cols}) values ({vals})'''
    cur.fast_executemany = True
    cur.executemany(query, df.values.tolist())

def delete_sql(cnxn, table):
    cnxn.execute(f'''delete {table}''')

def close_conn(cnxn):
    cnxn.close()

In [3]:
# cnxn = connect()
engine = connect_sqlalchemy()

In [4]:
def read_marsh():
    df = pd.read_csv('c:/tt/marsh_fabr.csv')
    df['sm'] = df['sm'].str.replace(' ', '')
    df['sm1'] = df['sm1'].str.replace(' ', '')
    df['rs'] = df['rs'].str.replace(' ', '')
    df['sm'].fillna('', inplace=True)
    df['sm1'].fillna('', inplace=True)
    return df

def to_devmarsh(df):
    ls = []
    for index, row in df.iterrows():
        bm = row['bm']
        id_m = row['id']
        sm = row['sm1']
        rs = row['rs']
        ls_str = sm.split(',')
        ls_rs = rs.split(',')
        ss = [[id_m, ls_rs[0], 'РЦ', 1, 0, 1], [id_m, bm, 'Светофор', 1, 0, 2]]
        i = 3
        for rs1 in ls_rs[1:]:
            ss.append([id_m, rs1, 'РЦ', 0, 1, i])
            i += 1
        for str1 in ls_str:
            if str1 != '':
                str2 = str1[1:]
                str3 = str1[:1]
                str_pos = 1
                if str3 == '-':
                    str_pos = 0
                ss.append([id_m, str2, 'Стрелка', str_pos, 0, 0])
        ls.extend(ss)
    df2 = pd.DataFrame(ls, columns=['marshrut_id', 'elm', 'telm', 'ppos', 'isp', 'pnisp'])
    df2.to_sql('devmarsh', engine, if_exists='append', index=False)
    # df2.to_sql('devmarsh', engine, if_exists='replace', index_label='id')
    # to_sql(cnxn, 'devmarsh', df2)

def to_sv(df):
    lsm = df['bm'].unique().tolist()
    dfm = pd.DataFrame(lsm, columns=['name'])
    dfm['pok'] = 0
    dfm['type'] = 0
    dfm['fault_B'] = 0
    dfm['fault_K'] = 0
    dfm['remont'] = 0
    dfm['marsh'] = 0
    dfm.to_sql('sv', engine, if_exists='append', index=False)
    # dfm.to_sql('sv', engine, if_exists='replace', index_label='id')
    # to_sql(cnxn, 'sv', dfm)

def to_str(df):
    ls = df['sm'].tolist()
    ls_str = []
    ls_str1 = []
    for l_st in ls:
        if l_st is not np.nan:
            for st in l_st.split(','):
                st1 = st.strip()
                st1 = st1[1:]
                if st1 not in ls_str:
                    ls_str.append(st1)
    for st1 in ls_str:
        s1 = st1
        s2 = s3 = ''
        if '/' in st1:
            ind = st1.index('/')
            s1 = st1[:ind]
            s2 = st1[ind+1:]
            s3 = st1
        ls_str1.append([s1,s2,s3])
        if s2 != '':
            ls_str1.append([s2,s1,s3])
    df_str = pd.DataFrame(ls_str1, columns=['name', 'name2', 'name_main'])
    df_str['pol'] = 0
    df_str['remont'] = 0
    df_str['marsh'] = 0
    df_str['mt_z'] = 0
    df_str['mt_p'] = 0
    df_str['mt_m'] = 0
    df_str.to_sql('str', engine, if_exists='append', index=False)
    # df_str.to_sql('str', engine, if_exists='replace', index_label='id')
    # to_sql(cnxn, 'str', df_str)

def to_rs(df):
    ls = df['rs'].tolist()
    ls_str = []
    for l_st in ls:
        if l_st is not np.nan:
            for st in l_st.split(','):
                st1 = st.strip()
                if st1 not in ls_str:
                    ls_str.append(st1)

    df_str = pd.DataFrame(ls_str, columns=['name'])
    df_str['state'] = 0
    df_str['remont'] = 0
    df_str['marsh'] = 0
    df_str.to_sql('rs', engine, if_exists='append', index=False)
    # df_str.to_sql('rs', engine, if_exists='replace', index_label='id')
    # to_sql(cnxn, 'rs', df_str)

def to_txt_ls(namefile, ls):
    path = path_start + namefile
    with open(path, "w", encoding='Windows-1251') as f:
        for ss in ls:
            f.write(ss +'\n')

def to_txt_ls2(namefile, ls):
    path = path_start + namefile
    with open(path, "w", encoding='Windows-1251') as f:
        for ss in ls:
            s1 = ''
            for ss1 in ss:
                s1 += str(ss1) + ','
            if len(s1) > 0:
                s1 = s1[:-1]
                f.write(s1)
                f.write('\n')

In [5]:
df = read_marsh()
dfm = pd.read_sql('''select * from sv''', engine)
dfstr = pd.read_sql('''select * from str''', engine)
dfrs = pd.read_sql('''select * from rs''', engine)
dfmarsh = pd.read_sql('''select * from devmarsh''', engine)

In [47]:
# cnxn.close()

In [51]:
# to_sv(df)
to_str(df)
to_rs(df)
to_devmarsh(df)

In [9]:
df1 = pd.read_sql('''select * from devmarsh''', engine)
df1

,id,marshrut_id,elm,telm,ppos,isp,pnisp
0,1,1,Т,РЦ,1,0,1
1,2,1,Мн,Светофор,1,0,2
2,3,1,1ГП,РЦ,0,1,3
3,4,2,Т,РЦ,1,0,1
4,5,2,Мз,Светофор,1,0,2
...,...,...,...,...,...,...,...
2199,2200,294,204-212,РЦ,0,1,3
2200,2201,294,203П,РЦ,0,1,4
2201,2202,294,212,Стрелка,0,0,0
2202,2203,294,204,Стрелка,1,0,0


In [84]:
def txt_sv_rs():
    df1 = dfmarsh
    ls = dfm['name'].tolist()
    ls1 = []
    for sv in ls:
        mt_id = df1[df1['elm']==sv]['marshrut_id'].iloc[0]
        rs1 = df1[(df1['marshrut_id']==mt_id) & (df1.telm == 'РЦ') & (df1.pnisp == 1)]['elm'].iloc[0]
        rs2 = df1[(df1['marshrut_id']==mt_id) & (df1.telm == 'РЦ') & (df1.pnisp == 3)]['elm'].iloc[0]
        ls1.append([sv, rs1, rs2])
    # to_txt_ls2("sv_rs.txt", ls1)
    return pd.DataFrame(ls1, columns=['sv', 'rs1', 'rs2'])
def txt_str_rs():
    df1 = pd.read_excel('marsh_fabr.xlsx', sheet_name='str_rs')
    ls1 = df1.values.tolist()
    to_txt_ls2("str_rs.txt", ls1)

def txt_rs_sv():
    dfsvrs = txt_sv_rs()
    dfsvrs = dfsvrs.astype({"rs1": "str", 'rs2':'str'})
    lsrs = dfrs.name.tolist()
    ls2 = []
    for rs in lsrs:
        ls3 = dfsvrs[dfsvrs['rs2']==rs]['sv'].tolist()
        ls2.append([rs] + ls3)
    to_txt_ls2("rs_sv.txt", ls2)

def txt_rs_str():
    dfstrrs = pd.read_excel('marsh_fabr.xlsx', sheet_name='str_rs')
    dfstrrs = dfstrrs.astype({"str": "str", 'rs':'str'})
    lsrs = dfrs.name.tolist()
    ls2 = []
    for rs in lsrs:
        ls3 = dfstrrs[dfstrrs['rs']==rs]['str'].tolist()
        ls2.append([rs] + ls3)
    to_txt_ls2("rs_str.txt", ls2)

In [85]:
# txt_sv_rs()
# txt_str_rs()
# txt_rs_sv()
# txt_rs_str()

In [ ]:
dfstrrs = pd.read_excel('marsh_fabr.xlsx', sheet_name='str_rs')
lsrs = dfrs.name.tolist()
ls2 = []
for rs in lsrs:
    ls3 = dfstrrs[dfstrrs['rs']==rs]['str'].tolist()
    ls2.append([rs] + ls3)

In [86]:
# x = dfstrrs['rs'].iloc[0]
dfstrrs[dfstrrs['rs']=='105-109']['str'].tolist()

['109']

In [76]:
dfstrrs['rs'].iloc[0]

1

In [80]:
dfstrrs = dfstrrs.astype({"str": "str", 'rs':'str'})

In [82]:
dfstrrs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   str     92 non-null     object
 1   rs      92 non-null     object
dtypes: object(2)
memory usage: 1.6+ KB


In [5]:
def get_dict_ru_en():
    s2 = """А а

A a

Л л

L l

Ц ц

Ts ts

Б б

B b

М м

M m

Ч ч

Ch ch

В в

V v

Н н

N n

Ш ш

Sh sh

Г г

G g

О о

O o

Щ щ

Shch shch

Д д

D d

П п

P p

Е е

E e

Р р

R r

Ы ы

Y y

Ж ж

Zh zh

С с

S s

З з

Z, z

Т т

T t

Э э

E e

И и

I i

У у

U u

Ю ю

Yu yu

Й й

Y y

Ф ф

F f

Я я

Ya ya

К к

K k

Х х

Kh kh

/ -

_ _"""
    ls1 = s2.split('\n\n')
    ls_ru = []
    cnt = len(ls1)
    for i in range(0,cnt,2):
        ls_ru = ls_ru + ls1[i].split(' ')

    ls_en = []
    cnt = len(ls1)
    for i in range(1,cnt,2):
        ls_en = ls_en + ls1[i].split(' ')

    ds = dict(zip(ls_ru,ls_en))
    return ds

ds_ru_en = get_dict_ru_en()

def ru_en(s1):
    s2 = ''
    for s_ru in str(s1):
        s_en = ds_ru_en.get(s_ru)
        if s_en:
            s2 += s_en
        else:
            s2 += s_ru
    return s2

def st_script_DDI(typ='rs',  open_mod='a', ls_name_dop=[]):
    dfsv1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    dfsv1['name'] = dfsv1['name'].astype(str)
    rezerv = 1
    var_typ = 'RS'
    if typ == 'str':
        var_typ = 'STR'
    with open(f'{path_start_kntr}script_ddi.txt', open_mod, encoding='Windows-1251') as f:
        for index, row in dfsv1.iterrows():
            rs_name = ru_en(row['name'])
            if row['name'] == 'резерв':
                rs_name += str(rezerv)
                rezerv += 1
            if len(ls_name_dop) != 0:
                n1 = int(row["n"])
                i = 0
                for name_dop in ls_name_dop:
                    n1 += i
                    st1 = f'E_{name_dop}_{var_typ}_{rs_name}:=EIO2_d{row["d"]}_r0_s{row["s"]}_DDI{row["ddi"]}02K.DIS_CH_IN[{n1}].VALUE;\n'
                    f.write(st1)
                    i += 1
            else:
                st1 = f'E_{var_typ}_{rs_name}:=EIO2_d{row["d"]}_r0_s{row["s"]}_DDI{row["ddi"]}02K.DIS_CH_IN[{row["n"]}].VALUE;\n'
                f.write(st1)

def st_script_Modbus_Read(typ='sv', open_mod='a'):
    dfstr1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    slave = 1
    rezerv = 1
    var_typ = 'BKC2'
    typ_name = 'SV'
    if typ == 'str':
        var_typ = 'PCPT'
        typ_name = 'STR'
    with open(f'{path_start_kntr}script_modbus_read.txt', open_mod, encoding='Windows-1251') as f:
        for index, row in dfstr1.iterrows():
            name = ru_en(row['name'])
            if row['name'] == 'резерв':
                name += str(rezerv)
                rezerv += 1

            st1 = f"MDB_{var_typ}_{name}" + r" ('0.0.3{192.168.11." + f"{row['sh']}" \
                  + r"}\\0." + f"{row['pol']}.{row['canal']}.{slave}', ARM_{typ_name}_{name});\n"
            f.write(st1)
            slave += 1

def var_xml(ls, start_name_var, typeName, open_mod='a'):
    with open(f'{path_start_kntr}var.txt', open_mod, encoding='Windows-1251') as f:
        for st in ls:
            st1 = f'        <variables name="{start_name_var}_{ru_en(st)}" typeName="{typeName}"></variables>\n'
            f.write(st1)

def var_xml_ARM(ls, typ, reg_start,  open_mod='a'):
    with open(f'{path_start_kntr}var.txt', "a", encoding='Windows-1251') as f:
        reg = reg_start
        ls_mw = []
        for st in ls:
            st1 = ru_en(st)
            s1 = f'''   <variables name="ARM_{typ}_{st1}" typeName="WORD" topologicalAddress="%MW{reg}">
            <attribute name="TimeStampSource" value="0"></attribute>
        </variables>
    '''
            ls_mw.append([st, reg, st1])
            reg += 1
            f.write(s1)
    return ls_mw

def kntr_write_data():

    lsstr = dfstr['name'].tolist()
    lsstr = lsstr[1:]
    lssv = dfm['name'].tolist()
    lsrs = dfrs['name'].tolist()
    lsrs = lsrs[1:]

    lsstr2 = []
    for index, row in dfstr[1:].iterrows():
        s1 = row['name']
        s3 = row['name_main']
        if s3 == '':
            lsstr2.append(s1)
        else:
            if s3 not in lsstr2:
                lsstr2.append(s3)

    # Переменные ----------------------------------------------------
    var_xml(lsrs, 'E_RS', 'EBOOL', 'w')
    var_xml(lsstr2, 'E_SD_STR', 'EBOOL')
    var_xml(lsstr2, 'E_OF_STR', 'EBOOL')
    var_xml(lssv, 'MDB_BKC2', 'BKC2_MODBUS_READ')
    var_xml(lsstr2, 'MDB_PCPT', 'PCPT_MODBUS_READ')

    ls_mw_str = var_xml_ARM(lsstr2, 'STR', 501)
    ls_mw_sv = var_xml_ARM(lssv, 'SV', 701)
    ls_mw_rs = var_xml_ARM(lsrs, 'RS', 901)

    # Регистры устройств exel----------------------------------------------------
    df1 = pd.DataFrame(ls_mw_sv, columns=['name', 'reg', 'name1'])
    df2 = pd.DataFrame(ls_mw_str, columns=['name', 'reg', 'name1'])
    df3 = pd.DataFrame(ls_mw_rs, columns=['name', 'reg', 'name1'])

    with pd.ExcelWriter(f'{path_start_kntr}reg_dev.xlsx') as writer:  # doctest: +SKIP
        df1.to_excel(writer, sheet_name='sv', index=False)
        df2.to_excel(writer, sheet_name='str', index=False)
        df3.to_excel(writer, sheet_name='rs', index=False)

    # EIO (st script)----------------------------------------------------
    st_script_DDI('rs', 'w')
    st_script_DDI('str', 'a', ['SD', 'OF'])

    # MODBUS READ BKC2 (st script)----------------------------------------------------
    st_script_Modbus_Read('sv', 'w')
    st_script_Modbus_Read('str', 'a')

In [11]:
    # dfsv1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='rs')
    # dfsv1['name'] = dfsv1['name'].astype(str)
    # rezerv = 1
    # with open(f'{path_start_kntr}script_ddi.txt', "w", encoding='Windows-1251') as f:
    #     for index, row in dfsv1.iterrows():
    #         rs_name = ru_en(ds, row['name'])
    #         if row['name'] == 'резерв':
    #             rs_name += str(rezerv)
    #             rezerv += 1
    #         st1 = f'E_RS_{rs_name}:=EIO2_d{row["d"]}_r0_s{row["s"]}_DDI{row["ddi"]}02K.DIS_CH_IN[{row["n"]}].VALUE;\n'
    #         f.write(st1)
    #
    # # STR EIO (st script)----------------------------------------------------
    # dfsv1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='str')
    # dfsv1['name'] = dfsv1['name'].astype(str)
    # rezerv = 1
    # with open(f'{path_start_kntr}script_ddi.txt', "a", encoding='Windows-1251') as f:
    #     for index, row in dfsv1.iterrows():
    #         rs_name = ru_en(ds, row['name'])
    #         if row['name'] == 'резерв':
    #             rs_name += str(rezerv)
    #             rezerv += 1
    #         st1 = f'E_STR_{rs_name}:=EIO2_d{row["d"]}_r0_s{row["s"]}_DDI{row["ddi"]}02K.DIS_CH_IN[{row["n"]}].VALUE;\n'
    #         f.write(st1)

   # MODBUS READ PCPT (st script)----------------------------------------------------
        # dfsv1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='sv')
    # slave = 1
    # rezerv = 1
    # with open(f'{path_start_kntr}script_modbus_read_bkc2.txt', "w", encoding='Windows-1251') as f:
    #     for index, row in dfsv1.iterrows():
    #         sv_name = ru_en(ds, row['name'])
    #         if row['name'] == 'резерв':
    #             sv_name += str(rezerv)
    #             rezerv += 1
    #
    #         st1 = f"MDB_BKC2_{sv_name}" + r" ('0.0.3{192.168.11." + f"{row['sh']}" \
    #               + r"}\\0." + f"{row['pol']}.{row['canal']}.{slave}', ARM_SV_{sv_name});\n"
    #         f.write(st1)
    #         slave += 1
    #
    # # MODBUS READ PCPT (st script)----------------------------------------------------
    # dfstr1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='str')
    # slave = 1
    # rezerv = 1
    # with open(f'{path_start_kntr}script_modbus_read_pcpt.txt', "w", encoding='Windows-1251') as f:
    #     for index, row in dfstr1.iterrows():
    #         str_name = ru_en(ds, row['name'])
    #         if row['name'] == 'резерв':
    #             str_name += str(rezerv)
    #             rezerv += 1
    #
    #         st1 = f"MDB_PCPT_{str_name}" + r" ('0.0.3{192.168.11." + f"{row['sh']+1}" \
    #               + r"}\\0." + f"{row['pol']}.{row['canal']}.{slave}', ARM_STR_{str_name});\n"
    #         f.write(st1)
    #         slave += 1

       # RS EBOOL ----------------------------------------------------
    # with open(f'{path_start_kntr}var.txt', "w", encoding='Windows-1251') as f:
    #     for st in lsrs:
    #         st1 = f'        <variables name="E_RS_{ru_en(ds, st)}" typeName="EBOOL"></variables>\n'
    #         f.write(st1)
    #
    #  # STR EBOOL ----------------------------------------------------
    # with open(f'{path_start_kntr}var.txt', "a", encoding='Windows-1251') as f:
    #     for st in lsstr2:
    #         st1 = f'        <variables name="E_STR_{ru_en(ds, st)}" typeName="EBOOL"></variables>\n'
    #         f.write(st1)
    #
    #     # BKC2_MODBUS_READ ----------------------------------------------------
    # with open(f'{path_start_kntr}var.txt', "a", encoding='Windows-1251') as f:
    #     for st in lssv:
    #         st1 = f'        <variables name="MDB_BKC2_{ru_en(ds, st)}" typeName="BKC2_MODBUS_READ"></variables>\n'
    #         f.write(st1)
    #
    #     # PCPT_MODBUS_READ ----------------------------------------------------
    # with open(f'{path_start_kntr}var.txt', "a", encoding='Windows-1251') as f:
    #     for st in lsstr2:
    #         st1 = f'        <variables name="MDB_PCPT_{ru_en(ds, st)}" typeName="PCPT_MODBUS_READ"></variables>\n'
    #         f.write(st1)

       #   # ARM WORD ----------------------------------------------------
    # with open(f'{path_start_kntr}var.txt', "a", encoding='Windows-1251') as f:
    #     #     STR---------------
    #     reg = 501
    #     for st in lsstr2:
    #         st1 = ru_en(ds, st)
    #         s1 = f'''   <variables name="ARM_STR_{st1}" typeName="WORD" topologicalAddress="%MW{reg}">
    #         <attribute name="TimeStampSource" value="0"></attribute>
    #     </variables>
    # '''
    #         ls_mw_str.append([st, reg, st1])
    #         reg += 1
    #         f.write(s1)
    #     #     SV---------------
    #     reg = 701
    #     for st in lssv:
    #         st1 = ru_en(ds, st)
    #         s1 = f'''   <variables name="ARM_SV_{st}" typeName="WORD" topologicalAddress="%MW{reg}">
    #         <attribute name="TimeStampSource" value="0"></attribute>
    #     </variables>
    # '''
    #         ls_mw_sv.append([st, reg, st1])
    #         reg += 1
    #         f.write(s1)
    #     #    RS---------------------
    #     reg = 901
    #     for st in lsrs:
    #         st1 = ru_en(ds, st)
    #         s1 = f'''   <variables name="ARM_RS_{st}" typeName="WORD" topologicalAddress="%MW{reg}">
    #         <attribute name="TimeStampSource" value="0"></attribute>
    #     </variables>
    # '''
    #         ls_mw_rs.append([st, reg, st1])
    #         reg += 1
    #         f.write(s1)

In [25]:
kntr_write_data()

In [ ]:
def st_script_Modbus_Read(typ='sv', open_mod='a'):
    dfstr1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    slave = 1
    rezerv = 1
    var_typ = 'BKC2'
    typ_name = 'SV'
    if typ == 'str':
        var_typ = 'PCPT'
        typ_name = 'STR'
    with open(f'{path_start_kntr}script_modbus_read.txt', open_mod, encoding='Windows-1251') as f:
        for index, row in dfstr1.iterrows():
            name = ru_en(row['name'])
            if row['name'] == 'резерв':
                name += str(rezerv)
                rezerv += 1

            st1 = f"MDB_{var_typ}_{name}" + r" ('0.0.3{192.168.11." + f"{row['sh']}" \
                  + r"}\\0." + f"{row['pol']}.{row['canal']}.{slave}', ARM_{typ_name}_{name});\n"
            f.write(st1)
            slave += 1

In [9]:
df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='sv')
df1

,name,sh,pol,canal
0,М1,6,1,0
1,М3,6,1,0
2,М5,6,1,0
3,М7,6,1,0
4,М9,6,1,0
...,...,...,...,...
150,М310,8,1,1
151,М312,8,1,1
152,М314,8,1,1
153,М316,8,1,1


In [17]:
def set_nameEn(path, typ):
    df1 = pd.read_excel(f'{path_start_kntr}{path}', sheet_name=typ)
    rezerv = 1
    ls1=[]
    for index, row in df1.iterrows():
        name = ru_en(row['name'])
        if row['name'] == 'резерв':
            name += str(rezerv)
            rezerv += 1
        ls1.append(name)
    df1['name_en'] = ls1
    return df1

def set_nameEns():
    df1 = set_nameEn('base/modbus_read_par.xlsx', 'sv')
    df2 = set_nameEn('base/modbus_read_par.xlsx', 'str')
    df3 = set_nameEn('base/modbus_read_par.xlsx', 'rs')
    with pd.ExcelWriter(f'{path_start_kntr}base/modbus_read_par.xlsx') as writer:  # doctest: +SKIP
        df1.to_excel(writer, sheet_name='sv', index=False)
        df2.to_excel(writer, sheet_name='str', index=False)
        df3.to_excel(writer, sheet_name='rs', index=False)

In [18]:
set_nameEns()

In [6]:
def get_ls_dev_modbus():
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='sv')
    sh=0
    pol=0
    cnl=0
    rezerv = 1
    ls1=[]
    ls2=[]
    ls3=[]
    sh1=pol1=cnl1=-1
    for index, row in df1.iterrows():

        sh = row['sh']
        pol = row['pol']
        cnl = row['canal']

        name = ru_en(row['name'])
        if row['name'] == 'резерв':
            name += str(rezerv)
            rezerv += 1

        s1 = f"{row['sh']},{row['pol']},{row['canal']}"
        if s1 in ls1:
            ls2.append(name)
        else:
            if sh1 != -1:
                ls3.append([[sh1, pol1, cnl1],ls2])
            ls1.append(s1)
            ls2 = [name]

        sh1 = row['sh']
        pol1 = row['pol']
        cnl1 = row['canal']

    ls3.append([[sh, pol, cnl],ls2])
    return ls3

In [8]:
ls1 = get_ls_dev_modbus()
print(ls1[0])

[[6, 1, 0], ['M1', 'M3', 'M5', 'M7', 'M9', 'M11', 'M13', 'M15', 'M17', 'M19', 'M21', 'M23', 'M25', 'M27', 'M29', 'M33', 'M35', 'M37']]


In [41]:
def st_var_Modbus_Read(typ='sv', open_mod='a'):
    slave=1
    with open(f'{path_start_kntr}t1.txt', 'w', encoding='Windows-1251') as f:
        for ls2, ls3 in ls1:
            name = ls3[0]
            name_var = f'MDB_ADR_SV_{ls3[0]}'
            adr = r"0.0.3{192.168.11." + f"{ls2[0]}" + r"}\\0." + f"{ls2[1]}.{ls2[2]}"

            i=1
            s1=f'		<variables name="MDB_ADR_SV_{name}" typeName="ARRAY[1..18] OF STRING">\n'
            f.write(s1)
            for sv in ls3:
                adr1 = f'{adr}.{slave}'
                s1=f'''		     <instanceElementDesc name="[{i}]">
                        <value>'{adr1}'</value>
                    </instanceElementDesc>
        '''
                f.write(s1)
                i+=1
                slave+=1
            s1 = '         </variables>\n'
            f.write(s1)

    with open(f'{path_start_kntr}t1.txt', 'a', encoding='Windows-1251') as f:
        for ls2, ls3 in ls1:
            s1 = f'        <variables name="MDB_BKC2_{ls3[0]}" typeName="MODBUS_RW_BKC2"></variables>\n'
            f.write(s1)



In [ ]:
<variables name="MDB_BKC2_M11" typeName="MODBUS_RW_BKC2"></variables>

In [93]:
dfrs[1:].to_excel(f'{path_start_kntr}rsss.xlsx')

In [106]:
dfsv1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='rs')
dfsv1['name'] = dfsv1['name'].astype(str)
ls1 = dfsv1['name'].tolist()


In [107]:
ls2 = []
for x in ls1:
    if x not in lsrs:
        ls2.append(x)
ls2

['резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'резерв',
 'Л1',
 'Л3',
 'Л5',
 'Л7',
 'резерв',
 '206ТП',
 'Л2',
 'Л4',
 'Л6',
 'Л8',
 'Л10']

In [6]:
with open(f'{path_start_kntr}script_modbus_bkc.txt', 'w', encoding='Windows-1251') as f:
        for i in range(1,19):
            s1 = f'''	IF cnt={i} AND NOT rdpar{i}[1].0  THEN
		READ_VAR (ADDMX(ADRS[cnt]), '%MW', 1, 2, rdpar{i}, datard);
		WORD_AS_BYTE (INT_TO_WORD(rdpar{i}[2]), bL, bH);
		err1:=BYTE_TO_INT(bL);
	END_IF;

'''
            f.write(s1)

        s1 = '\n----------------------------------------------------------'
        f.write(s1)

        for i in range(1,19):
            s1 = f'''	IF cnt={i} AND NOT wrpar{i}[1].0  THEN
		datawr1[1]:=WORD_TO_INT (DATAWR[cnt]);
		WRITE_VAR (ADDMX(ADRS[cnt]), '%MW', 1, 2, datawr1, wrpar{i});
		WORD_AS_BYTE (INT_TO_WORD(rdpar{i}[2]), bL, bH);
		err1:=BYTE_TO_INT(bL);
	END_IF;

'''
            f.write(s1)

In [10]:
with open(f'{path_start_kntr}script_modbus_bkc.txt', 'w', encoding='Windows-1251') as f:
        for i in range(1,19):
            s1 = f'''
{i}: IF NOT rdpar{i}[1].0  THEN
        READ_VAR (ADDMX(ADRS[cnt]), '%MW', 1, 2, rdpar{i}, datard);
        WORD_AS_BYTE (INT_TO_WORD(rdpar{i}[2]), bL, bH);
        err1:=BYTE_TO_INT(bL);
     END_IF;
'''
            f.write(s1)

        s1 = '\n----------------------------------------------------------\n'
        f.write(s1)

        for i in range(1,19):
            s1 = f'''
{i}: IF NOT wrpar{i}[1].0  THEN
        datawr1[1]:=WORD_TO_INT (DATAWR[cnt]);
        WRITE_VAR (ADDMX(ADRS[cnt]), '%MW', 1, 2, datawr1, wrpar{i});
        WORD_AS_BYTE (INT_TO_WORD(wrpar{i}[2]), bL, bH);
        err1:=BYTE_TO_INT(bL);
     END_IF;
'''
            f.write(s1)